In [29]:
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lagat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Define the training corpus
corpus = [
    "the dog saw a cat",
    "the dog chased the cat",
    "the cat climbed a tree"
]

In [32]:
# Remove stopwords using NLTK
stop_words = set(stopwords.words('english'))
corpus = [' '.join([word for word in sentence.split() if word.lower() not in stop_words]) for sentence in corpus]

In [15]:
#Create a vocabulary and a word-to-index dictionary
vocabulary = set(" ".join(corpus).split())
vocab_size = len(vocabulary)
w_to_index = {word: idx for idx, word in enumerate(vocabulary)}


In [16]:
# Define the one-hot encoding using dictionary
def one_hot(word, w_to_index):
    one_hot_vector = np.zeros(len(w_to_index))
    one_hot_vector[w_to_index[word]] = 1
    return one_hot_vector

In [17]:
# Initialize neural network parameters
input_size = len(w_to_index)
output_size = len(w_to_index)
hidden_size = 3

np.random.seed(0)
weights_input_hidden = np.random.randn(input_size, hidden_size)
biases_hidden = np.zeros(hidden_size)
biases_output = np.zeros(output_size)
weights_hidden_output = np.random.randn(hidden_size, output_size)
hidden_output = None

In [18]:
# forward pass function to calculate probabilities using softmax
def forward_pass(input_vector):
    global hidden_output
    hidden_input = np.dot(input_vector, weights_input_hidden) + biases_hidden
    hidden_output = 1 / (1 + np.exp(-hidden_input))
    output_input = np.dot(hidden_output, weights_hidden_output) + biases_output
    output_probabilities = softmax(output_input)
    return output_probabilities

In [19]:
# Softmax function
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=0, keepdims=True)

In [20]:
# Training loop (backpropagation)
learning_rate = 0.1
epochs = 1500


In [21]:
for epoch in range(epochs):
    for sentence in corpus:
        words = sentence.split()
        for i, target_word in enumerate(words[:-1]):
            input_word = words[i]
            one_hot_target_word = one_hot(words[i + 1], w_to_index)

            # Forward pass
            input_vector = one_hot(input_word, w_to_index)
            output_prob = forward_pass(input_vector)

            # Calculate loss
            loss = -np.log(output_prob.dot(one_hot_target_word))

            # Backpropagation
            delta_output = output_prob - one_hot_target_word
            delta_hidden = np.dot(delta_output, weights_hidden_output.T) * hidden_output * (1 - hidden_output)

            # Update weights and biases
            weights_hidden_output -= learning_rate * np.outer(hidden_output, delta_output)
            biases_output -= learning_rate * delta_output
            weights_input_hidden -= learning_rate * np.outer(input_vector, delta_hidden)
            biases_hidden -= learning_rate * delta_hidden

In [22]:

# Test the network by generating probabilities for all words in the vocabulary
target_word = 'cat'
input_vector_cat = one_hot(target_word, w_to_index)
output_prob = forward_pass(input_vector_cat)

In [23]:
# Print the probabilities for all words in the vocabulary
print("Probabilities for words following 'cat'")
for word, prob in zip(w_to_index.keys(), output_prob):
    print(f"{word}: {prob:.4f}")

Probabilities for words following 'cat'
cat: 0.0050
dog: 0.0002
tree: 0.0000
chased: 0.0020
saw: 0.0018
climbed: 0.9910
